# Árboles de regresión y random forest para regresión y clasificación

## Librerías a utilizar

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

## Carga de datasets

In [ ]:
mainpath = "./ds/"
train = "housing_train.csv"
test = "housing_test.csv"
df_train = pd.read_csv(mainpath + train)
df_test = pd.read_csv(mainpath + test)

## Resumen de datos

Las dimensiones del data frame, filas y columnas, se obtiene con la propiedad `shape`, los valores de las cabeceras se obtienen con la propiedad `columns.values`.

In [ ]:
df_train.shape

La función `describe()` devuelve el conteo de campos no nulos, media, desviación estándar y cuantiles para columnas númericas. En las columnas son objetos (categóricas) devolverá el conteo de campos no nulos, número de valores posibles, el valor más repetido y su frecuencia. Si se desea saber el tipo de datos que tienen las columnas se usa la propiedad `dtypes`.

In [ ]:
df_train.describe().transpose()

In [ ]:
df_train.describe(include='object').transpose()

In [ ]:
df_train.dtypes

## Matriz de correlación

In [ ]:
plt.figure(figsize=(20,8),dpi=80)
corrmat = df_train.corr()
sns.heatmap(corrmat, vmax=.8, annot=True)

## Valores perdidos

Trabajar con los valores perdidos requiere primero su ubicación, posteriormente se seleccionará que debe ser borrado y luego que debe ser sustituido con un nuevo valor, por supuesto habrá que decidir cual será dicho valor nuevo.

### Eliminar campos

Para ubicar si una celda tiene un valor vacío se usa la función `isnull()`, si se prefiere lógica inversa se usa `notnull`. Es posible  obtener un vector  de estos  valores con la  propiedad `values`, transformarlo  a un array con la función `ravel()` y sumar los valores verdaderos con la función `sum()`. También es posible obtener una lista ordenada de las columnas con más valores vacíos.

In [ ]:
df_train.isnull().sum().sort_values(ascending=False)

En  el ejemplo  de  arriba, el  valor es  el  número de  valores  vacíos, si  usamos la  función `notnull()` sería el número de valores no vacíos, la suma de ambos debe ser el número total de filas obtenido anteriormente.

Hay dos razones para la falta de valores en los data sets:

- Recolección de datos: No se consiguieron los datos.
- Extracción de datos: Los datos están en la  DB original pero no se extrajeron correctamente al data set.

Se deben evitar datos vacíos para no tener problemas de manejo de información. Se tienen dos opciones:

- Borrar las filas donde falten valores en alguna de las columnas
- Borrar las columnas donde no se tenga suficiente información

En nuestro caso es posible observar que las columnas `MiscFeature, Fence, PoolQC, FirePlaceQu y Alley` tienen muy pocos valores proporcionados (menos del 55 por ciento)y no vale la pena conservarlas. Otro criterio para asegurar que hacemos lo correcto es revisar las correlaciones con la columna `SalesPrice`.

In [ ]:
df_train.corr()['SalePrice'].sort_values()

Como el razonamiento es el correcto procederemos al borrado de columnas.

In [ ]:
def toDel(df):
    for col in df.columns.values:
        nv = pd.isnull(df[col]).values.ravel().sum()
        if nv > df.shape[0] * 0.45:
            print("Deleting: "+col)
            del df[col]
    return df

In [ ]:
df_train = toDel(df_train)

### Llenar campos

Es necesario detectar nuevamente que columnas tienen valores vacíos. Esta vez vamos a proceder a reemplazar esos valores. Como sabemos que hay valores númericos y categóricos vacíos; los numéricos serán reemplazados por el promedio original de la columna, los categóricos serán remplazados por el valor no nulo más cercano puede ser el valor que va antes (`ffill`) o el que va después (`bfill`), en este análisis será el segundo.

In [ ]:
def DetectNull(df):
    candidates = []
    for col in df.columns.values:
        nv = pd.isnull(df[col]).values.ravel().sum()
        if nv > 0:
            candidates.append((col, df[col].dtype, nv))
    return candidates

In [ ]:
def FillNull(df, list):
    for col in list:
        if col[1] == 'float64':
            df[col[0]] = df[col[0]].fillna(df[col[0]].mean())
        else:
            df[col[0]] = df[col[0]].fillna(method="bfill")
    return df

In [ ]:
df_train = FillNull(df_train, DetectNull(df_train))
df_train.isnull().sum().sort_values(ascending=False)

## Creación de categorías de SalesPrice

In [ ]:
def SalePriceGroupValue(x):
    if x >= 500001:
        return 3
    elif x <= 100000:
        return 1
    return 2

In [ ]:
df_train["SalePriceGroup"] = df_train["SalePrice"].apply(SalePriceGroupValue)

In [ ]:
df_train.tail()